<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/doubt_marker_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [24]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [25]:

%cd /content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/3_Classification

%ls

/content/drive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/3_Classification
doubt_marker_accuracy.gsheet       gold_standard_doubt_marker_100.csv
doubt_marker_classification.ipynb  predictions.csv


In [26]:
df = pd.read_csv("gold_standard_doubt_marker_100.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sentence_id   100 non-null    int64 
 1   sentence      100 non-null    object
 2   matched_term  100 non-null    object
 3   doubt_label   100 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [27]:

df['label'] = df['doubt_label']
df.drop(['sentence_id','doubt_label'], axis=1)
del df['sentence_id'], df['doubt_label']
df.head(3)

,sentence,matched_term,label
0,"Mom is ambivalent about pumping, and does not ...",\bambivalent\b,1
1,PT ADAMANTLY EXPRESSED THIS IS NOT WHAT SHE WA...,\badamantly\b,0
2,believes th Amiodarone extreme tremors Precaut...,\bbelieves\b,1


In [28]:
cols = ['sentence', 'matched_term']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df.head(3)

,sentence,matched_term,label,text
0,"Mom is ambivalent about pumping, and does not ...",\bambivalent\b,1,"Mom is ambivalent about pumping, and does not ..."
1,PT ADAMANTLY EXPRESSED THIS IS NOT WHAT SHE WA...,\badamantly\b,0,PT ADAMANTLY EXPRESSED THIS IS NOT WHAT SHE WA...
2,believes th Amiodarone extreme tremors Precaut...,\bbelieves\b,1,believes th Amiodarone extreme tremors Precaut...


In [29]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 5e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

In [30]:
class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        #print(self.targets.shape)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [31]:
newdf = df[['text','label']]
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    100 non-null    object
 1   label   100 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [32]:
train_size = 0.8
train_data=newdf.sample(frac=train_size,random_state=0)
test_data=newdf.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(newdf.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (100, 2)
TRAIN Dataset: (80, 2)
TEST Dataset: (20, 2)


In [33]:
train_data['label'].unique()

array([0, 1])

In [34]:
test_data['label'].unique()

array([1, 0])

In [35]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [36]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2) #output labels = 1 for binary classification

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [37]:
model = RobertaClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [38]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [39]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [40]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [41]:
for epoch in range(EPOCHS):
    train(epoch)


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss: 0.6865119934082031
Training Accuracy: 62.5



1it [00:02,  1.31it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

2it [00:03,  1.84s/it]
3it [00:04,  1.32s/it]
4it [00:04,  1.08s/it]
5it [00:05,  1.06it/s]
6it [00:07,  1.15it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In t

The Total Accuracy for Epoch 0: 60.0
Training Loss Epoch: 0.6775977194309235
Training Accuracy Epoch: 60.0



0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

1it [00:01,  1.38it/s]

...

The Total Accuracy for Epoch 8: 97.5
Training Loss Epoch: 0.08503743829205632
Training Accuracy Epoch: 97.5



0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

1it [00:01,  1.51it/s]

Training Loss: 0.5045774579048157
Training Accuracy: 87.5


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

2it [00:01,  1.03it/s]
3it [00:02,  1.18it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave m

The Total Accuracy for Epoch 9: 96.25
Training Loss Epoch: 0.15196020230650903
Training Accuracy Epoch: 96.25


In [42]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [43]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss=0
    nb_tr_steps=0
    nb_tr_examples=0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples

    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu


Initial Run -- one interation

In [44]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)



0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

1it [00:00,  4.54it/s]

Validation Loss: 0.4068194031715393
Validation Accuracy: 87.5


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

2it [00:01,  1.46it/s]
3it [00:01,  2.20it/s]


Validation Loss Epoch: 1.422150234381358
Validation Accuracy Epoch: 70.0
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        14
           1       0.50      0.50      0.50         6

    accuracy                           0.70        20
   macro avg       0.64      0.64      0.64        20
weighted avg       0.70      0.70      0.70        20

Confusion Matrix:
[[11  3]
 [ 3  3]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(510), tensor(90), tensor(34...         0          0
1  [tensor(0), tensor(3320), tensor(326), tensor(...         0          1
2  [tensor(0), tensor(26145), tensor(4467), tenso...         0          0
3  [tensor(0), tensor(438), tensor(15597), tensor...         1          1
4  [tensor(0), tensor(10975), tensor(12606), tens...         0          0



0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

1it [00:03,  4.53it/s]

Validation Loss: 2.1170034408569336
Validation Accuracy: 62.5


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(

2it [00:03,  2.01s/it]
3it [00:03,  1.21s/it]


Validation Loss Epoch: 1.6977717752257984
Validation Accuracy Epoch: 70.0
              precision    recall  f1-score   support

           0       0.79      0.79      0.79        14
           1       0.50      0.50      0.50         6

    accuracy                           0.70        20
   macro avg       0.64      0.64      0.64        20
weighted avg       0.70      0.70      0.70        20

Confusion Matrix:
[[11  3]
 [ 3  3]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(2571), tensor(18685), tenso...         1          0
1  [tensor(0), tensor(448), tensor(3937), tensor(...         0          0
2  [tensor(0), tensor(510), tensor(90), tensor(10...         0          0
3  [tensor(0), tensor(3320), tensor(326), tensor(...         0          1
4  [tensor(0), tensor(2515), tensor(38295), tenso...         0          0
test accuracy = 70.00%


In order to account for variations in GPU states, we ran each of the models 10 times and provide the mean accuracy


## Predictions

In [46]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)
        token_type_ids = torch.tensor([inputs["token_type_ids"]], dtype=torch.long).to(device)

        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)

        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return df


In [61]:
predicted_classes = predict_on_dataframe(newdf, model, tokenizer, max_len=128)
print(predicted_classes)

annotation_drew_sample = pd.read_csv("annotation_Drew_sample_doubt_markers.csv")
annotation_drew_sample.info()
annotation_drew_sample= annotation_drew_sample.rename(columns={'Sentence':'text'})

predicted_mega = predict_on_dataframe(annotation_drew_sample, model, tokenizer, max_len=128)
predicted_mega
#
filtered_df = predicted_mega[predicted_mega['predictions'] == 1]

filtered_df

,Sentence ID,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,...,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,text,regex,matched_term,doubt_testimony,annotator_comments,predictions
9,10619325,681531,83982,147681.0,2145-05-03,2145-05-03 06:14:00,2145-05-03 13:58:48,Physician,Physician Resident/Attending Progress Note - MICU,19692.0,...,2145-05-04,2145-05-04,2145-05-04,1,Chief Complaint: 24 Hour Events: - Family insi...,insists|insisting|reinsists|insist|insisted|\....,\binsisted\b,NaN,NaN,1
12,13725111,5518,11229,165559.0,2168-12-15,NaN,NaN,Discharge summary,Report,NaN,...,NaN,NaN,NaN,0,Insisted on IV pain meds and tx to [**Hospital...,insists|insisting|reinsists|insist|insisted|\....,\binsisted\b,NaN,NaN,1
13,7036819,28167,32425,121264.0,2160-07-11,NaN,NaN,Discharge summary,Report,NaN,...,2160-08-28,2160-08-28,2160-08-28,1,The patient insisted on going to her appt rath...,insists|insisting|reinsists|insist|insisted|\....,\binsisted\b,NaN,NaN,1
21,15908648,1652110,30252,135623.0,2130-06-18,2130-06-18 03:40:00,2130-06-18 03:55:00,Nursing/other,Report,16866.0,...,NaN,NaN,NaN,0,Consider weaning Dobuta/Lasix infusion Consid...,"chat_gpt|""skeptical|dubiousness|doubting|skept...",\bdoubtful\b,NaN,NaN,1
32,12334361,462575,96923,189534.0,2167-04-26,2167-04-26 12:20:00,2167-04-26 19:07:14,Physician,Physician Resident Admission Note,21284.0,...,NaN,NaN,NaN,0,- Change abx to cipro IV 400mg q12h and metro...,insists|insisting|reinsists|insist|insisted|\....,\bbelieves\b,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,8588356,733589,65675,145965.0,2133-02-22,2133-02-22 04:07:00,2133-02-22 04:46:30,Nursing,Nursing Progress Note,21336.0,...,2133-02-23,2133-02-23,NaN,1,"Action: Fent increased to 300mcg, mult vent c...",insists|insisting|reinsists|insist|insisted|\....,\bbelieves\b,NaN,NaN,1
483,3776052,329517,27455,133225.0,2143-07-27,2143-07-27 08:01:00,2143-07-27 11:53:57,Physician,Physician Resident Progress Note,16760.0,...,2143-07-30,2143-07-30,2143-07-30,1,"-FiO2 down to 30% overnight, now back up to 4...",insists|insisting|reinsists|insist|insisted|\....,\bbelieves\b,NaN,NaN,1
485,6802990,24307,5060,174823.0,2182-04-07,NaN,NaN,Discharge summary,Report,NaN,...,NaN,NaN,NaN,0,He has a history of IV heroin and smoking coca...,insists|insisting|reinsists|insist|insisted|\....,\binsisted\b,NaN,NaN,1
490,106610,1466270,15716,158787.0,2147-01-11,2147-01-11 04:59:00,2147-01-11 05:19:00,Nursing/other,Report,18469.0,...,2147-08-30,2147-08-30,2147-09-04,1,Plan: continue mechanical ventilation and tri...,insists|insisting|reinsists|insist|insisted|\....,\bbelieves\b,NaN,NaN,1


In [ ]:

# constant for classes
#classes = ('Neither Source/Location', 'Source, no user', 'User, no source', 'User and source')


#print(test_data["label"])
#predicted_roberta = model.
#predict(test_data["text"])
#print(predicted)
# Build confusion matrix
#cf_matrix = confusion_matrix(test_data["label"], predicted)
#df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
#                     columns = [i for i in classes])
#plt.figure(figsize = (4,4))
#sn.heatmap(df_cm, annot=True)
#plt.savefig('output.png')

Notes for next time-- want to turn this script into a function where I can pass through the hyperparameters.

In [ ]:
# svm baseline

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.svm import SVC

#from sklearn.pipeline import Pipeline
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto'))])
#text_clf.fit(train_data['text'], train_data['label'])

In [ ]:
#docs_test = test_data['text']
#predicted = text_clf.predict(docs_test)
#np.mean(predicted == test_data['label'])

In [ ]:
#test_accs = []
#for i in range(10):
#  model = RobertaClass()
#  model.to(device)
#
#  loss_function = torch.nn.CrossEntropyLoss()
#  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
#
#  for epoch in range(EPOCHS):
#      train(epoch)
#  acc = valid(model, testing_loader)
#  #print("test accuracy = %0.2f%%" % acc)
#  test_accs.append(acc)

# mean_acc = sum(test_accs) / len(test_accs)
# print("MEAN ACCURACY = %0.2f%%" % mean_acc)
# output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/d/d9/Wilhelm_Scream.ogg").play()')
